In [1]:
import itertools
import IPython.display as disp

import numpy as np
from sympy import Indexed, simplify, collect, O, log, I, diff, eye, solve
from sympy.solvers.solveset import linear_eq_to_matrix
from symengine import expand, Symbol, conjugate, symbols, Matrix, linsolve, Add

# Normal Form for Spatial Dynamics in the Swift-Hohenberg Equation
In this notebook we repreoduce the paper `Normal Form for Spatial Dynamics in the Swift-Hohenberg Equation` by John Burke and Edgar Knobloch. We consider steady solutions of the system
$$
  \frac{\partial u}{\partial t} = \mu u - (1 + \partial_{x}^{2})^{2} u + f(u),
$$
where typically $f(u) = n_{2} u^{2} + n_{3} u^{3}$ or $f(u) = n_{3} u^{3} + n_{5} u^{5}$. With $\partial / \partial t \equiv 0$ we can write the above equation at the bifurcation point $\mu = 0$ as
$$
  \frac{dU}{dx} = \mathcal{L} U + \mathcal{N},
$$
where
$$
 U = \begin{pmatrix}
       u_{0} \\
       u_{1} \\
       u_{2} \\
       u_{3}
     \end{pmatrix}, \quad
 \mathcal{L} = \begin{pmatrix}
                 0 & 1 & 0 & 0 \\
                 0 & 0 & 1 & 0 \\
                 0 & 0 & 0 & 1 \\
                 - 1 & 0 & -2 & 0
                \end{pmatrix}, \quad
  \mathcal{n} = \begin{pmatrix}
                  0 \\
                  0 \\
                  0 \\
                  n_{2} u^{2} + n_{3} u^{3}
                \end{pmatrix}.
$$

In [2]:
# linear matrix
L = Matrix([[0, 1, 0, 0],
            [0, 0, 1, 0],
            [0, 0, 0, 1],
            [-1, 0, -2, 0]])
L

[0, 1, 0, 0]
[0, 0, 1, 0]
[0, 0, 0, 1]
[-1, 0, -2, 0]

The system isn reversible under spatial reflection ($x \rightarrow -x$, $u \rightarrow u$); we write this symmetry $R$ acting on the vector $U$ as

$$
  R =\begin{pmatrix}
       1 & 0 & 0 & 0 \\
       0 & -1 & 0 & 0 \\
       0 & 0 & 1 & 0 \\
       0 & 0 & 0 & -1
     \end{pmatrix}.
$$

In [3]:
R = eye(4)
for i in range(1, 3):
    R[2 * i - 1, 2 * i - 1] = -1

We introduce a cooridnate transform from the $U$ basis to $(A, B, \bar{A}, \bar{B})$, defined as
$$
  A = A(U) = \sum_{n_{i} \geq 0} A_{n_{0} n_{1} n_{2} n_{3}} u_{0}^{n_{0}} u_{1}^{n_{1}} u_{2}^{n_{2}} u_{3}^{n_{3}},
$$
$$
  B = B(U) = \sum_{n_{i} \geq 0} B_{n_{0} n_{1} n_{2} n_{3}} u_{0}^{n_{0}} u_{1}^{n_{1}} u_{2}^{n_{2}} u_{3}^{n_{3}},
$$
and using the reversibility symmetry the conjugate counterparts, namely
$$
  \bar{A} = \bar{A}(U) = \sum_{n_{i} \geq 0} A_{n_{0} n_{1} n_{2} n_{3}} u_{0}^{n_{0}} (-u_{1})^{n_{1}} u_{2}^{n_{2}} (-u_{3})^{n_{3}},
$$
$$
  \bar{B} = \bar{B}(U) = - \sum_{n_{i} \geq 0} B_{n_{0} n_{1} n_{2} n_{3}} u_{0}^{n_{0}} (-u_{1})^{n_{1}} u_{2}^{n_{2}} (-u_{3})^{n_{3}},
$$

In [4]:
# main variables
A = Symbol('A')
B = Symbol('B')

A_bar = Symbol('A_bar')  # conjugate(A)
B_bar = Symbol('B_bar')  # conjugate(B)

z_tilde = Matrix([A, B, A_bar, B_bar])

relations = {conjugate(A): A_bar, conjugate(B): B_bar,
             conjugate(A_bar): A, conjugate(B_bar): B}

In [5]:
u0, u1, u2, u3 = symbols('u0 u1 u2 u3')
U = [u0, u1, u2, u3]
RU = [u0, -u1, u2, -u3]

In [6]:
powers = [(0, 1, 2, 3) for _ in range(4)]
poly_powers = [el for el in itertools.product(*powers) if sum(el) in [1, 2, 3]]
poly_powers
poly = 4 * [0]
unknowns = []
for idx in poly_powers:
    coeff = Symbol('A_{}'.format(''.join([str(el) for el in idx])))
    unknowns.append(coeff)
    poly[0] += coeff * np.product([x**i for x, i in zip(U, idx)])
    coeff = Symbol('B_{}'.format(''.join([str(el) for el in idx])))
    unknowns.append(coeff)
    poly[1] += coeff * np.product([x**i for x, i in zip(U, idx)])
    coeff = Symbol('A_{}'.format(''.join([str(el) for el in idx])))
    unknowns.append(coeff)
    poly[2] += coeff * np.product([x**i for x, i in zip(RU, idx)])
    coeff = Symbol('B_{}'.format(''.join([str(el) for el in idx])))
    unknowns.append(coeff)
    poly[3] -= coeff * np.product([x**i for x, i in zip(RU, idx)])

In [7]:
simplify(poly[0])

A_0001*u3 + A_0002*u3**2 + A_0003*u3**3 + A_0010*u2 + A_0011*u2*u3 + A_0012*u2*u3**2 + A_0020*u2**2 + A_0021*u2**2*u3 + A_0030*u2**3 + A_0100*u1 + A_0101*u1*u3 + A_0102*u1*u3**2 + A_0110*u1*u2 + A_0111*u1*u2*u3 + A_0120*u1*u2**2 + A_0200*u1**2 + A_0201*u1**2*u3 + A_0210*u1**2*u2 + A_0300*u1**3 + A_1000*u0 + A_1001*u0*u3 + A_1002*u0*u3**2 + A_1010*u0*u2 + A_1011*u0*u2*u3 + A_1020*u0*u2**2 + A_1100*u0*u1 + A_1101*u0*u1*u3 + A_1110*u0*u1*u2 + A_1200*u0*u1**2 + A_2000*u0**2 + A_2001*u0**2*u3 + A_2010*u0**2*u2 + A_2100*u0**2*u1 + A_3000*u0**3

The normal form for this bifurcation is given as
$$
  \frac{dA}{dx} = iA + B + i A P(\mu; c_{1}, c_{3}),
$$
$$
  \frac{dB}{dx} = iB + i B P(\mu; c_{1}, c_{3}) + A Q(\mu; c_{1}, c_{3}),
$$
where
$c_{1} = \lvert A \rvert^{2}$ and $c_{3} = i (A \bar{B} - \bar{A} B) / 2$. The polynomials $P$ and $Q$ are given by
$$
P(\mu; c_{1}, c_{2}) = p_{1} \mu + p_{2} c_{1} + p_{3} c_{3} + p_{4} c_{1}^{2} + p_{5} c_{1} c_{3} + p_{6} c_{3}^{2}
$$
and
$$
Q(\mu; c_{1}, c_{2}) = -q_{1} \mu + q_{2} c_{1} + q_{3} c_{3} + q_{4} c_{1}^{2} + q_{5} c_{1} c_{3} + q_{6} c_{3}^{2}.
$$

In [8]:
# integrals
c1 = A * A_bar
c2 = I * B / A + log(A)
c3 = I * (A * B_bar - A_bar * B) / 2

In [9]:
p2, p3, p4, p5, p6 = symbols('p2 p3 p4 p5 p6')
q2, q3, q4, q5, q6 = symbols('q2 q3 q4 q5 q6')

In [10]:
P = p2 * c1 + p3 * c3# + p4 * c1**2 + p5 * c1 * c3 + p6 * c3**2
Q = q2 * c1 + q3 * c3# + q4 * c1**2 + q5 * c1 * c3 + q6 * c3**2

In [11]:
simplify(P)

A*A_bar*p2 + I*p3*(A*B_bar - A_bar*B)/2

In [12]:
n2, n3 = symbols('n2 n3', real=True)

In [13]:
expansions = {A: poly[0], B: poly[1], A_bar: poly[2], B_bar: poly[3]}

To assist with separating orders we introduce an additional scaling $\epsilon$. This scaling is not related to the large length scale $X$ which appears in the weakly nonlinear analyses.

In [14]:
epsilon = Symbol('epsilon')

In [15]:
scalings = {u0: epsilon * u0, u1: epsilon * u1, u2: epsilon * u2, u3: epsilon * u3}

Using the chain rule we rewrite the normal form as
$$
  \begin{split}
  \frac{dA}{dx} &= \left(\frac{\partial A}{\partial u_{0}} \frac{d u_{0}}{dx} + \frac{\partial A}{\partial u_{1}} \frac{d u_{1}}{dx} + \frac{\partial A}{\partial u_{2}} \frac{d u_{2}}{dx} + \frac{\partial A}{\partial u_{3}} \frac{d u_{3}}{dx}\right) \\
  &= \left(\frac{\partial A}{\partial u_{0}} u_{1} + \frac{\partial A}{\partial u_{1}} u_{2} + \frac{\partial A}{\partial u_{2}} u_{3} + \frac{\partial A}{\partial u_{3}} (- u_{0} - 2 u_{2} + n_{2} u_{0}^{2} + n_{3} u_{0}^{3}) \right) \\
  &= iA + B + i A P(\mu; c_{1}, c_{3}),
  \end{split}
$$
$$
  \begin{split}
    \frac{dB}{dx} &= \left(\frac{\partial B}{\partial u_{0}} \frac{d u_{0}}{dx} + \frac{\partial B}{\partial u_{1}} \frac{d u_{1}}{dx} + \frac{\partial B}{\partial u_{2}} \frac{d u_{2}}{dx} + \frac{\partial B}{\partial u_{3}} \frac{d u_{3}}{dx}\right) \\
  &= \left(\frac{\partial B}{\partial u_{0}} u_{1} + \frac{\partial B}{\partial u_{1}} u_{2} + \frac{\partial B}{\partial u_{2}} u_{3} + \frac{\partial B}{\partial u_{3}} (- u_{0} - 2 u_{2} + n_{2} u_{0}^{2} + n_{3} u_{0}^{3}) \right) \\
  &= iB + i B P(\mu; c_{1}, c_{3}) + A Q(\mu; c_{1}, C_{3}).
  \end{split}
$$
We can also derive additional equations using $d \bar{A} / dx$ and $d \bar{B} / dx$, but having already enforced the reversibility symmetry in the expansions of $A$ and $B$ these additional equations are redundant.

In [16]:
eqn1 = expand(diff(poly[0], u0) * u1 + diff(poly[0], u1) * u2 + diff(poly[0], u2) * u3 + diff(poly[0], u3) * (-u0 - 2 * u2 + n2 * u0**2 + n3 * u0**3) - I * A - B - I * A * P)
eqn1 = expand(eqn1.subs(expansions).subs(scalings))

In [17]:
eqn2 = expand(diff(poly[1], u0) * u1 + diff(poly[1], u1) * u2 + diff(poly[1], u2) * u3 + diff(poly[1], u3) * (-u0 - 2 * u2 + n2 * u0**2 + n3 * u0**3) - I * B - I * B * P - A * Q)
eqn2 = expand(eqn2.subs(expansions).subs(scalings))

## n = 1
At order $n = 1$, we collect together terms linear in the scaling term $\epsilon$, which gives us a system of 8 equations in 8 unknowns $\{A_{1000}, A_{0100}, A_{0010}, A_{0001}\}$ and $\{B_{1000}, B_{0100}, B_{0010}, B_{0001}\}$.

In [18]:
tmp1 = collect(eqn1, [epsilon], evaluate=False)

In [19]:
system1 = collect(tmp1[epsilon._sympy_()], U, evaluate=False)

In [20]:
tmp2 = collect(eqn2, [epsilon], evaluate=False)

In [21]:
system2 = collect(tmp2[epsilon._sympy_()], U, evaluate=False)

In [22]:
system = []
coeffs = []
for k, v in system1.items():
    system.append(v)
    coeffs.extend(v.free_symbols)

In [23]:
for k, v in system2.items():
    system.append(v)
    coeffs.extend(v.free_symbols)

In [24]:
coeffs = list(set(coeffs))

In [25]:
# order coefficients
def coefficient_sort(coeffs):
    r"""
    Given a set of coefficients of the form $A_{ijkl}$, $B_{ijkl}$,
    sort those coefficients firstly alphabetically, then in reverse
    numerical order for the subscripts.
    
    Arguments:
        coeffs: list of coefficients to be sorted
    """
    shuffle = []
    for term in ('A', 'B'):
        part_coeffs = [(i, str(coeff)) for i, coeff in enumerate(coeffs) if str(coeff).startswith(term)]
        part_sorted = sorted(part_coeffs, key=lambda s: s[1].split('_')[1], reverse=True)
        shuffle.extend(t[0] for t in part_sorted)
    return [coeffs[i] for i in shuffle]

coeffs = coefficient_sort(coeffs)

In [26]:
# convert to matrix form
M, RHS = linear_eq_to_matrix(system, coeffs)

In [27]:
M.LUsolve(RHS)

NonInvertibleMatrixError: Matrix det == 0; not invertible.

From above we see that the matrix $M$ is rank deficient. In this case this means that some of the equations are linearly dependent. The rank of the matrix is

In [28]:
M.rank()

6

Computing the left-nullspace of the matrix $M$ thus yiekds two vectors

In [29]:
K = M.nullspace()

In [30]:
for vec in K:
    disp.display(vec)

Matrix([
[I],
[1],
[I],
[1],
[0],
[0],
[0],
[0]])

Matrix([
[ -1],
[2*I],
[  1],
[  0],
[  I],
[  1],
[  I],
[  1]])

In [31]:
M.rref()

(Matrix([
 [1, 0, 0, -I, 0, 0, 0,    1],
 [0, 1, 0, -1, 0, 0, 0, -2*I],
 [0, 0, 1, -I, 0, 0, 0,   -1],
 [0, 0, 0,  0, 1, 0, 0,   -I],
 [0, 0, 0,  0, 0, 1, 0,   -1],
 [0, 0, 0,  0, 0, 0, 1,   -I],
 [0, 0, 0,  0, 0, 0, 0,    0],
 [0, 0, 0,  0, 0, 0, 0,    0]]), (0, 1, 2, 4, 5, 6))

In [32]:
alpha, beta = symbols('alpha beta')
sol_values = alpha * K[0] + beta * K[1]
sol = {}
for coeff, val in zip(coeffs, sol_values):
    sol[coeff] = val
sol

{A_1000: I*alpha - beta,
 A_0100: alpha + 2*I*beta,
 A_0010: I*alpha + beta,
 A_0001: alpha,
 B_1000: I*beta,
 B_0100: beta,
 B_0010: I*beta,
 B_0001: beta}

This gives, as expected, a two-parameter family of solutions (owing to the rank deficiency of $M$). The form is not quite the same as given in the paper, but is equivalent under a change of variables $\alpha = \alpha(\xi_{1}, \xi_{2})$ and $\beta = \beta(\xi_{1}, \xi_{2})$.

In [33]:
xi1, xi2 = symbols(r'\xi_{1}, \xi_{2}')
bk_answer = [(2 * xi1 + xi2) / 4,  # A_1000
             -I * (3 * xi1 + xi2) / 4,  # A_0100
             xi2 / 4,  # A_0010
             -I * (xi1 + xi2) / 4,  # A_0001
             -I * xi1 / 4,  # B_1000
             -xi1 / 4,  # B_0100
             -I * xi1 / 4,  # B_0010
             -xi1 / 4]  # B_0001
transform_system = []
for k, ans in zip(coeffs, bk_answer):
    transform_system.append(sol[k] - ans)

In [34]:
transform = solve(transform_system, [alpha, beta], dict=True)[0]

In [35]:
for k, v in sol.items():
    sol[k] = simplify(v.subs(transform))

In [36]:
for k, v in sol.items():
    disp.display(k, v)

A_1000

\xi_{1}/2 + \xi_{2}/4

A_0100

-I*(3*\xi_{1} + \xi_{2})/4

A_0010

...

-\xi_{1}/4

B_0010

-I*\xi_{1}/4

B_0001

-\xi_{1}/4

In this case we can also use the `solve` method from sympy to recover the coefficients

## n = 2
We now proceed to second order, collecting together terms quadratic in $\epsilon$. At this order we recover the coefficients $\{A_{2000}, A_{0200}, A_{0020}, A_{0002}, A_{1100}, A_{1010}, A_{1001}, A_{0110}, A_{0101}, A_{0011}\}$ and $\{B_{2000}, B_{0200}, B_{0020}, B_{0002}, B_{1100}, B_{1010}, B_{1001}, B_{0110}, B_{0101}, B_{0011}\}$.

In [37]:
system1 = expand(tmp1[epsilon._sympy_()**2].subs(sol))

In [38]:
system2 = expand(tmp2[epsilon._sympy_()**2].subs(sol))

In [39]:
system = []
coeffs = []

In [40]:
for _, term in collect(system1, [np.product(el) for el in itertools.product(U, U)], evaluate=False).items():
    system.append(term)
    coeffs.extend(term.free_symbols)

In [41]:
for _, term in collect(system2, [np.product(el) for el in itertools.product(U, U)], evaluate=False).items():
    system.append(term)
    coeffs.extend(term.free_symbols)

In [42]:
coeffs = list(set(coeffs).intersection([s._sympy_() for s in unknowns]))
coeffs = coefficient_sort(coeffs)

In [43]:
M, RHS = linear_eq_to_matrix(system, coeffs)

In [44]:
_ = M.LUsolve(RHS)

In [45]:
sol2 = {coeff: simplify(val) for coeff, val in zip(coeffs, _)}

In [46]:
for coeff, val in sol2.items():
    disp.display(coeff, val)

A_2000

n2*(-18*\xi_{1} + 7*\xi_{2})/108

A_1100

I*n2*(19*\xi_{1} - 23*\xi_{2})/54

A_1010

...

-5*I*\xi_{1}*n2/54

B_0011

-\xi_{1}*n2/27

B_0002

I*\xi_{1}*n2/27

# n = 3
Moving on to order 3, we now recover the set of coefficients $\{A_{ijkl}: i + j + k + l = 3\}$ and $\{B_{ijkl}: i + j + k + l = 3\}$. The normal form coefficients $\{p_{2}, p_{3}, q_{2}, q_{3}\}$ are also recovered at this order.

In [47]:
system1 = expand(tmp1[epsilon._sympy_()**3].subs(sol).subs(sol2))

In [48]:
system2 = expand(tmp2[epsilon._sympy_()**3].subs(sol).subs(sol2))

In [49]:
system = []
coeffs = []

In [50]:
for _, term in collect(system1, [np.product(el) for el in itertools.product(U, U, U)], evaluate=False).items():
    system.append(term)
    coeffs.extend(term.free_symbols)

In [51]:
for _, term in collect(system2, [np.product(el) for el in itertools.product(U, U, U)], evaluate=False).items():
    system.append(term)
    coeffs.extend(term.free_symbols)

In [52]:
coeffs = list(set(coeffs).intersection([s._sympy_() for s in unknowns]))
coeffs = coefficient_sort(coeffs)

In [53]:
M, RHS = linear_eq_to_matrix(system, coeffs)

Looking at the rank of the matrix $M$ we see that we should expect a four paramater family of solutions.

In [54]:
M.rank()

36

In [64]:
K = M.T.nullspace()

In [56]:
alpha, beta, gamma, delta = symbols('alpha beta gamma delta')
sol_values = alpha * K[0] + beta * K[1] + gamma * K[2] + delta * K[3]
sol3 = {}
for coeff, val in zip(coeffs, sol_values):
    sol3[coeff] = val
sol3

{A_3000: -I*alpha + 2*I*beta - 2*delta + gamma,
 A_2100: -alpha + 2*beta + 4*I*delta - 2*I*gamma,
 A_2010: -I*alpha + 4*I*beta - gamma,
 A_2001: -alpha + 2*beta,
 A_1200: -2*I*alpha + 3*I*beta - 3*delta + 2*gamma,
 A_1110: 2*beta + 4*I*delta,
 A_1101: -2*I*alpha + 4*I*beta - 4*delta + 2*gamma,
 A_1020: I*alpha + 2*I*beta + 2*delta - gamma,
 A_1011: 2*beta,
 A_1002: I*beta - delta,
 A_0300: -2*alpha + 3*beta + 6*I*delta - 4*I*gamma,
 A_0210: -2*I*alpha + 3*I*beta + 3*delta - 2*gamma,
 A_0201: -4*alpha + 7*beta + 8*I*delta - 4*I*gamma,
 A_0120: alpha + 2*I*gamma,
 A_0111: -2*I*alpha + 4*I*beta + 4*delta - 2*gamma,
 A_0102: -2*alpha + 5*beta + 2*I*delta,
 A_0030: I*alpha + gamma,
 A_0021: alpha,
 A_0012: I*beta + delta,
 A_0003: beta,
 B_3000: 2*I*delta - I*gamma,
 B_2100: 2*delta - gamma,
 B_2010: 4*I*delta - I*gamma,
 B_2001: 2*delta - gamma,
 B_1200: 3*I*delta - 2*I*gamma,
 B_1110: 2*delta,
 B_1101: 4*I*delta - 2*I*gamma,
 B_1020: 2*I*delta + I*gamma,
 B_1011: 2*delta,
 B_1002: I*delta

We are more interested in the solution of the coefficients of the normal form, which are recovered by multiplying the kernel by the non-zero RHS of the system at 3rd order.

In [65]:
M_kern = Matrix([list(k) for k in K])

In [66]:
param_system = simplify(M_kern * RHS)

In [67]:
free_coefficients = [p2._sympy_(), p3._sympy_(), q2._sympy_(), q3._sympy_()]

In [68]:
Mfree, RHSfree = linear_eq_to_matrix(param_system, free_coefficients)

In [69]:
sol3_free = Mfree.LUsolve(RHSfree)

In [70]:
for coeff, val in zip(free_coefficients, sol3_free):
    disp.display(coeff, simplify(val))

p2

-(374*\xi_{1}*n2**2 + 243*\xi_{1}*n3 + 228*\xi_{2}*n2**2 + 162*\xi_{2}*n3)/(432*\xi_{1}**3)

p3

-(72*\xi_{1}*n2**2 + 130*\xi_{2}*n2**2 + 81*\xi_{2}*n3)/(81*\xi_{1}**3)

q2

-(38*n2**2 + 27*n3)/(36*\xi_{1}**2)

q3

-(82*\xi_{1}*n2**2 + 81*\xi_{1}*n3 + 684*\xi_{2}*n2**2 + 486*\xi_{2}*n3)/(216*\xi_{1}**3)

In [72]:
for coeff, val in zip(free_coefficients, sol3_free):
    disp.display(coeff, simplify(val).subs({xi1: 1, xi2: 0}))

p2

-187*n2**2/216 - 9*n3/16

p3

-8*n2**2/9

q2

-19*n2**2/18 - 3*n3/4

q3

-41*n2**2/108 - 3*n3/8